In [1]

In [1]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  

# 나는 스스로 판단할 수 있는 많은 정보를 얻기 위해 EARLY_STOPPING_ROUNDS를 높게 설정합니다.(OPTIMIZE_ROUNDS가 설정된 경우)
# 만약 실제로 조기 종료를 원한다면 EARLY_STOPPING_ROUNDS를 줄여야합니다.

처음에는 MAX_ROUNDS를 상당히 높게 설정하고 OPTIMIZE_ROUNDS를 사용하여 적당한 라운드 수를 찾는것을 추천합니다.
(제 판단에는, 모든 폴드 사이에서 best_ntree_limit의 최댓값에 가깝게 해야합니다. 모델이 적절히 정규화 되었거나
verbose=True를 설정하고, 모든 폴드에서 잘 동작하는 라운드 수를 찾기 위한 세부 정보를 본다면 더 높을 수도 있습니다.)
그 다음 OPTIMIZE_ROUNDS를 중지하고, MAX_ROUND를 적절한 총 라운드 수로 설정하세요.

각 폴드마다 최적의 라운드를 선택하는 "조기 종료" 방식은 검증 데이터에 과적합하는 문제가 존재합니다.
따라서 테스트 데이터를 예측하기 위한 최적의 모델을 생성하지 못할 수 있고,
다른 모델과 스태킹/앙상블을 위해 검증 데이터를 생성하는 경우 앙상블에 너무 많은 weight를 갖게 할 것입니다.
또 다른 문제점은(XGBoost의 기본 값은) 가장 좋은 라운드보다 조기 종료가 발생한 라운드(개선되지 않고 정체되있는 라운드)를 사용하는 것입니다.
정체되는 구간이 충분히 길다는 가정이 있다면 이것은 과적합 문제를 해결해줍니다. 그러나 지금까지는 도움이 되지 않았습니다.
(모든 폴드에 대해 일정한 라운드로 진행한것보다 폴드 당 20라운드 조기 종료를 한 것이 검증 점수가 안좋았다. 그래서 조기 종료는 실제로는 underfit 처럼 보였다.)

In [2]

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc

In [3]

def gini(list_of_values):

    sorted_list = sorted(list_of_values)
    
    height, area = 0, 0
    
    for value in sorted_list:
    
        height += value
        
        area += height - value / 2.
        
    fair_area = height * len(list_of_values) / 2.
    
    return (fair_area - area) / fair_area

In [3]:
## gini 계산
## 참고한 링크: https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true) # numba가 이해할 수 있는 형식으로 변환 
    y_true = y_true[np.argsort(y_prob)] # y_prob 기준으로 오름차순 정렬
    ntrue = 0 
    gini = 0
    delta = 0
    n = len(y_true) # y_true의 길이 n
    for i in range(n-1, -1, -1): # y_prob가 큰 수 부터
        y_i = y_true[i] # y_true의 값을 순서대로 y_i에 저장
        ntrue += y_i # y_i의 값을 ntrue라는 변수에 더함
        gini += y_i * delta # y_i * delta값을 gini 변수에 더함
        delta += 1 - y_i # 1 - y_i값을 delta 값으로 초기화
    gini = 1 - 2 * gini / (ntrue * (n - ntrue)) # 반복을 통해 나온 ntrue와 gini값을 이용하여 gini 결과를 구함
    return gini

In [4]

머신 러닝 모델을 평가하는 데 사용하기 위해 label과 prediction을 입력으로 받고 계산된 계수를 리턴한다.

label을 정렬한 prediction의 인덱스로 재배열했을 때의 지니계수를 구하고, 그 값을 실제값의 지니계수 값으로 나눈 normalized gini coefficient를 사용한다.

예측값으로 정렬하여 계산한 지니계수와 원본값의 지니계수를 비교하는 것으로 동일할 경우 1.0을 리턴한다.

In [4]:
# 함수는 olivier 커널에서 가져왔습니다.
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

# 원본과 prediction의 누적값이 동일할 때의 1.0의 값이 가장 크므로, loss로 사용하기 위해 음수로 변경해서 일치시 가장 작은 값을 갖게 한다.
def gini_xgb(preds, dtrain):
    labels = dtrain.get_label() # 범례를 추출하여 labels에 저장
    gini_score = -eval_gini(labels, preds) # eval_gini 함수를 구해서 -를 붙인 뒤 gini_score에 저장
    return [('gini', gini_score)] # gini_score return

# series의 길이와 noise_level을 이용하여 noise를 추가해주는 add_noise 함수 정의
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series))) # 노이즈 추가
  
def target_encode(trn_series=None, # 범주형 train 피처
                  val_series=None, # 범주형 
                  tst_series=None, # 범주형 test 피처
                  target=None, # 타겟 피처
                  min_samples_leaf=1, # 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing은 Daniele Micci-Barreca의 논문에서와 같이 계산됩니다.
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : pd.Series 형태의 학습할 범주형 피처
    tst_series : pd.Series 형태의 테스트할 범주형 피처
    target : pd.Series 형태의 타겟 데이터
    min_samples_leaf (int) : 범주의 평균을 고려할 최소 샘플
    smoothing (int) : 범주 평균과 이전의 균형을 맞추기 위한 스무딩 효과 
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1) # train series와 target을 concat하여 temp에 저장
    # Compute target mean
    # trn_series의 name 피처를 groupby한 것에서 target의 name피처의 평균과 count값 구함
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    #
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    # averages에서 mean이랑 count는 drop
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [5]

In [5]:
# Read data
train_df = pd.read_csv('./input/train.csv', na_values="-1") # na_values에 들어있는 값들은 csv파일을 불러올 때 자동으로 nan값으로 변경된다.
test_df = pd.read_csv('./input/test.csv', na_values="-1")

In [6]

feature selection (Boruta 알고리즘)을 하였을 때, shadow feature 보다 feature importance가 높은 경우만 train_features에 저장.

이외의 feature는 상대적으로 관련 없을 거라고 판단되어 drop 시킴

In [6]:
# olivier 커널 참고
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]
# 조합 추가
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [7]

In [8]:
# 데이터 처리
id_test = test_df['id'].values # test DF의 id 피처의 값을 id_test라는 변수에 저장
id_train = train_df['id'].values # train DF의 id 피처의 값을 id_train라는 변수에 저장
y = train_df['target'] # train DF의 target 피처의 값을 y에 저장

start = time.time() # 시작 시간 측정
for n_c, (f1, f2) in enumerate(combs): # combs를 enumerate 하여 n_c (인덱스 번호) 반복
    name1 = f1 + "_plus_" + f2 # 피처 두 개의 이름을 plus 붙여서 합침
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='') # current feature : 피처 이름, index, 걸린 시간
    print('\r' * 75, end='')
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x)) # 두 피처의 값을 더해서 새로운 피처 생성. train DF에 저장
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x)) # 두 피처의 값을 더해서 새로운 피처 생성. test DF에 저장
    # Label Encode (문자열 값을 숫자형 카테고리 값으로 변환)
    lbl = LabelEncoder() # p.119 LabelEncoder 객체 생성
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values)) # fit과 transform으로 레이블 인코딩 수행
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1) # train_features에 해당 피처들 저장
    
X = train_df[train_features] # 전체 train DF에서 해당 피처 값들만 X에 저장
test_df = test_df[train_features] # 전체 test DF에서 해당 피처 값들만 test_df에 저장

f_cats = [f for f in X.columns if "_cat" in f] # cat(범주형) 변수들만 f_cats에 저장

current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.0

In [8]

In [9]:
y_valid_pred = 0*y #  y에 저장한 train DF의 target 피처의 값에 0을 곱해 (0으로 만들어) y_valid_pred에 저장
y_test_pred = 0 # y_test_pred를 0으로 초기화

In [9]

In [10]:
# 폴드 설정 p.103 (교차 검증)
K = 5 # K값 5로 설정. 5개의 폴드 세트 즉. 5개의 예측 평가 구할 예정 (5등분)
kf = KFold(n_splits = K, random_state = 1, shuffle = True) # n_splits값을 5로 설정하여 KFold 객체 생성
np.random.seed(0)

In [10]

n_estimators : 반복 수행하려는 트리의 개수

max_depth : 트리 최대 깊이

objective : 최솟값을 가져야할 손실함수를 정의 (이중분류/다중분류)

learning_rate : 부스팅 스텝을 반복적으로 수행할 때 업데이트되는 학습률 값

subsample : 트리가 커져서 과적합되는 것을 제어하기 위해 데이터를 샘플링하는 비율을 지정

min_child_weight : 트리에서 추가적으로 가지를 나눌지를 결정하기 위해 필요한 데이터들의 weight 총합

colsample_bytree : max_features와 유사. 트리 생성에 필요한 피처를 임의로 샘플링 하는데 사용

scale_pos_weight : 특정 값으로 치우친 비대칭한 클래스로 구성된 데이터 세트의 균형을 유지하기 위한 파라미터

gamma : 트리의 리프 노드를 추가적으로 나눌지를 결정할 최소 손실 감소

reg_alpha : L1 regulation 제어를 위한 값 과적합 제어를 위한 것

reg_lambda : L2 regulation 제어를 위한 값 과적합 제어를 위한 것

In [11]:
## 분류기 설정
model = XGBClassifier(    
                        n_estimators=MAX_ROUNDS,
                        max_depth=4,
                        objective="binary:logistic", # 이진분류
                        learning_rate=LEARNING_RATE, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=10,
                        reg_alpha=8,
                        reg_lambda=1.3,
                     )

In [11]

In [12]:
# CV 실행
for i, (train_index, test_index) in enumerate(kf.split(train_df)): # 만들어 두었던 kfold 객체로 train DF을 쪼개고 하나씩 실행
    
    # 폴드에 대한 데이터 생성
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index] # train_index와 text_index를 기준으로 y_train과 y_valid로 나눔
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy() # train_index와 test_index를 기준으로 X를 X_train, X_valid로 나눔
    X_test = test_df.copy() # X_test에는 test DF 전체
    print( "\nFold ", i)
    
    # 데이터 인코드
    for f in f_cats: # 넣어놓았던 범주형 피처 하나씩 반복 (타겟 인코딩 실행)
        X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                        trn_series=X_train[f],
                                                        val_series=X_valid[f],
                                                        tst_series=X_test[f],
                                                        target=y_train,
                                                        min_samples_leaf=200,
                                                        smoothing=10,
                                                        noise_level=0
                                                        )
    # 폴드에 대한 모델 실행
    if OPTIMIZE_ROUNDS:
        eval_set=[(X_valid,y_valid)]
        fit_model = model.fit( X_train, y_train, 
                               eval_set=eval_set,
                               eval_metric=gini_xgb,
                               early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                               verbose=False
                             )
        print( "  Best N trees = ", model.best_ntree_limit ) # 최적의 ntree_limit과 
        print( "  Best gini = ", model.best_score ) # 최적의 지니계수 출력
    else:
        fit_model = model.fit( X_train, y_train )
        
    # 폴드에 대한 검증 예측 생성
    pred = fit_model.predict_proba(X_valid)[:,1] # 실제 예측값
    print( "  Gini = ", eval_gini(y_valid, pred) ) # 결과값과 실제 예측값 비교
    y_valid_pred.iloc[test_index] = pred # 실제 예측값을 test_index에 맞게 y_valid_pred에 저장
    
    # 테스트 세트의 예측을 누적
    y_test_pred += fit_model.predict_proba(X_test)[:,1] # y_test_pred에는 그 에측값을 누적시킴
    
    del X_test, X_train, X_valid, y_train
    
y_test_pred /= K  # 테스트 세트 예측의 평균

print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


Fold  0


c:\users\user\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:58:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\AppData\Local\Temp/ipykernel_15112/3932066299.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "eval_gini" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\User\AppData\Local\Temp/ipykernel_15112/3932066299.py (6)

File "C:\Users\User\AppData\Local\Temp\ipykernel_15112\3932066299.py", line 6:
<source missing, REPL/exec in use?>

  @jit
C:\Users\User\AppData\Local\Temp/ipykernel_15112/3932066299.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "eval_gini" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "C:\Users\User\AppData\Local\Temp\ipykernel_15112\3932066299.py", line 12:
<source missing, REPL/exec in use?>

  @jit
c:\users\user\miniconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "eval_gini" was compiled in obj

  Gini =  0.2851059728784705

Fold  1


c:\users\user\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:59:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  Gini =  0.28185495483845957

Fold  2
[15:00:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  Gini =  0.27429910138514

Fold  3
[15:01:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behav

C:\Users\User\AppData\Local\Temp/ipykernel_15112/3932066299.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "eval_gini" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\User\AppData\Local\Temp/ipykernel_15112/3932066299.py (6)

File "C:\Users\User\AppData\Local\Temp\ipykernel_15112\3932066299.py", line 6:
<source missing, REPL/exec in use?>

  @jit
C:\Users\User\AppData\Local\Temp/ipykernel_15112/3932066299.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "eval_gini" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "C:\Users\User\AppData\Local\Temp\ipykernel_15112\3932066299.py", line 12:
<source missing, REPL/exec in use?>

  @jit
c:\users\user\miniconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "eval_gini" was compiled in obj

0.28501477642381845

In [12]

In [ ]:
# 스태킹 / 앙상블을위한 검증 예측 저장
val = pd.DataFrame() # val이라는 DF 생성
val['id'] = id_train # id_train에 저장해둔 id 값들을 새로운 id피처에 저장
val['target'] = y_valid_pred.values # y_valid_pred의 값들을 새로운 target 피처에 저장
val.to_csv(DATA_PATH + 'xgb_valid.csv', float_format='%.6f', index=False) # csv파일로 저장

In [13]

In [ ]:
## 제출 파일 생성
sub = pd.DataFrame() # sub이라는 DF 생성
sub['id'] = id_test # id_test에 저장해둔 id 값들을 새로운 id피처에 저장
sub['target'] = y_test_pred # y_test_pred의 값들을 새로운 target 피처에 저장
sub.to_csv(DATA_PATH + 'xgb_submit.csv', float_format='%.6f', index=False) # csv파일로 저장